In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as pyplot
import os
import re
import math
import tifffile
from enum import Enum

In [ ]:
ASSETS_FOLDER = "assets/"
OUTPUT_FOLDER = "output/"

In [ ]:
class Index(Enum):
  EVI = 'evi',
  NDVI = 'ndvi',
  GNDVI = 'gndvi',
  MSI = 'msi',
  NDWI = 'ndwi',
  NDBI = 'ndbi',
  NDMI = 'ndmi'

In [ ]:
def setup_dictionary(folder):
  images = {}
  for filename in os.listdir(folder):
    #img = cv2.imread(os.path.join(folder, filename))
    key = re.search('B[0|1|8][0-9|A]', filename)
    #if img is not None and key:
    if key:
      #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      #img = img / 255
      #img[img > 1] = 1
      #img[img <= 0] = 0.0001

      #img = img.astype(np.float32)

      #if(img.shape[0] > max_size):
      #  max_size = img.shape[0]
      
      images[key.group(0)] = filename

      print('handled:', key.group(0))
  return images

In [ ]:
def fix_image(image):
  image[image > 1] = 1
  image[image <= 0] = 0
  return image

In [ ]:
def load_image(filename):
  image = cv2.imread(ASSETS_FOLDER + filename)

  if image is not None:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)
    image = image / 255
    image = fix_image(image)

    return image

In [ ]:
def upscale_image(image, size, factor):
  if factor == 1:
    return image
    
  print('upscaling by: ' + str(factor))
  image = cv2.resize(image, (size * factor, size * factor))
  return image

In [ ]:
def upscale_images(images):
  shapes = list(map(lambda image: image.shape[0], images))
  max_size = max(shapes)

  for id, image in enumerate(images):
    images[id] = upscale_image(image, image.shape[0], math.floor(max_size / image.shape[0]))
    images[id] = fix_image(images[id])

  return images

In [ ]:
def display_image(image):
  imgplot = pyplot.imshow(image)
  pyplot.axis('off')
  pyplot.show()

In [ ]:
def calculate_index(images, index):
  print('calculating:', index)

  if index == Index.EVI:
    loaded_images = [load_image(images['B08']), load_image(images['B04']), load_image(images['B02'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + 6 * loaded_images[1] - 7.5 * loaded_images[2] + 1

    print('0 max', np.amax(loaded_images[0]))
    print('0 min', np.amin(loaded_images[0]))
    print('1 max', np.amax(loaded_images[1]))
    print('1 min', np.amin(loaded_images[1]))
    print('2 max', np.amax(loaded_images[2]))
    print('2 min', np.amin(loaded_images[2]))
    print('top max', np.amax(numerator))
    print('top min', np.amin(numerator))
    print('top max', np.amax(denominator))
    print('top min', np.amin(denominator))
    
    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDVI:
    loaded_images = [load_image(images['B08']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.GNDVI:
    loaded_images = [load_image(images['B08']), load_image(images['B03'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.MSI:
    loaded_images = [load_image(images['B11']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)

    return np.divide(loaded_images[0], loaded_images[1], out=np.zeros_like(loaded_images[0]), where=loaded_images[1]!=0)
  elif index == Index.NDWI:
    loaded_images = [load_image(images['B03']), load_image(images['B11'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDBI:
    loaded_images = [load_image(images['B11']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDMI:
    loaded_images = [load_image(images['B09']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  else:
    return None

In [ ]:
images = setup_dictionary(ASSETS_FOLDER)

In [ ]:
#print(max_size)
#upscale_images(images, max_size)

for key in images:
  image = load_image(images[key])
  print(key + ': ' + images[key] + ' ' + str(image.shape))
  
  display_image(image)

In [ ]:
for index in Index:
  index_image = calculate_index(images, index)

  #print(index_image.shape)
  if index_image is not None:
    print(np.amax(index_image))
    print(np.amin(index_image))

    print(index_image)
    
    #index_image = index_image * 255
    #index_image[index_image > 255] = 255
    #index_image[index_image < 0] = 0
    #index_image = index_image.astype(np.uint8)

    #display_image(index_image)

    print('saving to:', OUTPUT_FOLDER + index.name + ".tif")

    #cv2.imwrite(OUTPUT_FOLDER + index.name + ".tif", index_image)
    tifffile.imwrite(OUTPUT_FOLDER + index.name + ".tif", index_image, photometric="rgb")